In [ ]:
import os
import pandas_gbq
import requests
import pandas as pd
import numpy as np
# import win32com.client as win32
from pathlib import Path
from datetime import datetime
import csv
import xlsxwriter
from xlsxwriter.utility import xl_col_to_name
import json
import warnings
import time

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.precision", 2)

api_key = "YNSTeozZP99BsrTsoN0Ly9uy78lQdpKu3KoiRILa"
username = "apiusermc.universalmusic@umusic.com"
password = "zYhyiJq1dkPP"

In [ ]:
def get_current_global_week():
    current_date = datetime.now().strftime('%Y-%m-%d')

    weeks = pandas_gbq.read_gbq(
        """
            SELECT day, week FROM `umg-finance.metadata.day`
        """,
        project_id='umg-us'
        )
    weeks['day'] = pd.to_datetime(weeks['day']).dt.strftime('%Y-%m-%d')
    weeks_dict = weeks.set_index('day')['week'].to_dict()
    current_global_week = weeks_dict[current_date] - 1

    return current_global_week


In [ ]:
# AUTHORIZE
def get_authorize_token(username=username, password=password):
    headers = {"Content-Type": "application/x-www-form-urlencoded", "x-api-key": api_key}
    payload = {"username":username, 'password': password}

    url = "https://api.musicconnect.mrc-data.com/auth"
    r = requests.post(
        url= url, 
        headers=headers, 
        data=payload
    )

    access_token = r.json()['access_token']

    return access_token

In [ ]:
def get_chart(global_week, access_token, chart_id =  'CHRISTIANAIRPLAY'):

    # data in post contains global week to pull
    data = {
        "week_id": global_week
        }

    headers = {
                'Content-Type':'application/json',
                'Authorization': access_token,
                'x-api-key': api_key,
                'Accept':'application/vnd.mrc-data.chart.v1+json'
            }
            
    url = f'https://api.musicconnect.mrc-data.com/api/chart/data/{chart_id}'

    r = requests.post(
        url=url, 
        headers=headers, 
        data=json.dumps(data)
        )
    # print(r.json())
    
    if r.status_code == 200:

        df = pd.DataFrame([pd.Series(row) for row in r.json()])

        dict_for_df = {}

        for k, v in df.iteritems():
            # print(k)
            if k == 'title_id' or k == 'artist_id' or k == 'label':
                dict_for_df[k] = v
            if k == 'metrics':
                for i in v:
                    for k in i:
                        if not dict_for_df.__contains__(k['name']):
                            dict_for_df[k['name']] = [k['value']]
                        else:
                            dict_for_df[k['name']].append(k['value'])

        # load data to dataframe
        df = pd.DataFrame(dict_for_df)
        df['title_id'] = df['title_id'].replace(r'^\s*$', np.nan, regex=True)

        df = df[df['title_id'].notnull()]
        df = df.set_index(['title_id'])
        df.columns = pd.MultiIndex.from_product([[chart_id.lower()], df.columns.str.lower()])

        return df.head(100)
    
    print(r)


In [ ]:
def get_data_by_title_id(title_id, access_token):

    # GET ARTIST META DATA BY Neilsen ID

    
    headers = {
        'Content-Type':'application/json',
        'Authorization': access_token,
        'x-api-key': api_key,
        # 'Accept':'application/vnd.mrc-data.chart.v1.1+json'
    }

    url = f"https://api.musicconnect.mrc-data.com/api/song/{title_id}/data"
    r = requests.post(
        url= url, 
        headers=headers, 
    )

    title_id_dict = {
        'title_id': [],
        'top_level': [],
        'period': [],
        'value': []
    }


    for list_i in r.json()['metrics']:
        # print(list_i['name'])

        for i, list_k in enumerate(list_i['value']):
            # print(i, list_k)
            title_id_dict['title_id'].append(title_id)
            title_id_dict['top_level'].append(list_i['name'].lower())

            for k, v in list_k.items():
                # print(k, v)
                if k == 'name':
                    title_id_dict['period'].append(v)
                    # print(v)
        
                elif k == 'value':
                    title_id_dict['value'].append(v)
                    # print(v)
            


    df_row = pd.DataFrame(title_id_dict).pivot_table(index='title_id', columns=['top_level', 'period'], values='value', aggfunc='first')
    time.sleep(1)

    return df_row

In [ ]:
def get_artist_data(chart_df, access_token):

    df = pd.DataFrame()

    for i, (k, v) in enumerate(chart_df.iterrows()):
        # print(i)
        while True:
            try:
                df = pd.concat([df, get_data_by_title_id(k, access_token)])
                time.sleep(1)
                # print(f'{i}: {v[3]} {v[4]}')
            except KeyError as E:
                print(f'title_id: {k}, song: {v[3]}, artist: {v[4]}, ERROR:{E}\n\n')
                continue
            break

    try:
        df_out = pd.concat([df, chart_df], 1)
        return df_out

    except Exception as E:
        print(E)
        return df

In [ ]:
def drop_extra_and_sort_df(df_in):
    df = (
            df_in.drop(df_in.columns[[0]], axis=1)
            .set_index([('christianairplay', 'rank'), ('christianairplay', 'label'),  ('christianairplay', 'artist'),  ('christianairplay', 'song')], drop=True)
        )

    for value in df.columns.get_level_values(0).unique():
        if value == 'christianairplay':

            # get percent change for airplay
            df[(value,'% change audience')] = df.loc[:, [(value, 'lp audience'), (value, 'tp audience')]].pct_change(axis=1).loc[:, (value, 'tp audience')].replace([np.inf, -np.inf], 1)
            
            # remove uneeded columns from chart df
            df.drop(columns=[
                ('christianairplay', 'tp spins'),
                ('christianairplay', '+/- chg. spins'),
                ('christianairplay', 'lp spins'),
                ('christianairplay', 'stations'),
                ('christianairplay', 'adds'),
                ('christianairplay', 'lp audience'),
                ('christianairplay', '+/- chg. audience'),
                ], 
                inplace=True)
    
        else:
            # gert percent change for the rest of consumptionm
            df[(value,'% change')] = df.loc[:, [(value, 'lp'), (value, 'tp')]].pct_change(axis=1).loc[:, (value, 'tp')].replace([np.inf, -np.inf], 1)

            ## remove unneeded columns from chart artist consumption data
            ## DONT KNOW WHATS GOING ON: 'digital tracks' top level throws error when trying to remove these columns, hence try: except:
            try: df.drop(labels= (value, 'lp'), inplace=True)
            except: pass
            try: df.drop(labels= (value, 'atd'), inplace=True)
            except: pass
            try: df.drop(labels= (value, 'ytd'), inplace=True)
            except: pass
            df[(value, 'tp')] = df[(value, 'tp')].fillna(0).astype(int)

    df = df.reset_index()

    col_order = (
        ('christianairplay', 'artist'),
        ('christianairplay', 'song'),
        ('christianairplay', 'rank'),
        # ('christianairplay', 'label'),
        ('christianairplay', 'tp audience'),
        ('christianairplay', '% change audience'),  
        ('song w/ses on-demand', 'tp'),
        ('song w/ses on-demand', '% change'),
        # ('song w/ses on-demand audio', 'tp'),
        # ('song w/ses on-demand audio', '% change'),
        ('streaming on-demand audio', 'tp'),
        ('streaming on-demand audio', '% change'),
        ('streaming on-demand video', 'tp'),
        ('streaming on-demand video', '% change'),
        ('streaming on-demand total', 'tp'),
        ('streaming on-demand total', '% change'),
        # ('christianairplay', 'tp stream'),
        ('digital tracks', 'tp'),
        ('digital tracks', '% change'),
    )

    # sort columns
    df = df.reindex(columns=pd.MultiIndex.from_tuples(col_order))
    df = df.fillna(0)

    format_columns = [
        'ARTIST',
        'SONG',
        'BILLBOARD AIRPLAY CHART RANK',
        'AUDIENCE',
        'AUDIENCE CHANGE',
        'CONSUMPTION',
        'CONSUMPTION CHANGE',
        'AUDIO STREAMS',
        'AUDIO STREAMS CHANGE',
        'VIDEO STREAMS',
        'VIDEO STREAMS CHANGE',
        'TOTAL STREAMS',
        'TOTAL STREAMS CHANGE',
        'DIGITIAL TRACKS',
        'DIGITIAL TRACKS CHANGE'
        ]


    df = df.droplevel(axis=1, level=0)

    df.columns = format_columns

    # sorting because some song_id's are empty in chart_df, therefore the artist row is dropped and sorting order is off (if rank < 100 are at bottom of data)
    df = df.sort_values('BILLBOARD AIRPLAY CHART RANK')
    
    return df


In [ ]:
def format_and_write_to_excel(df, out_path):

    sheet_name = 'radio_report'
    workbook = xlsxwriter.Workbook(str(out_path))
    worksheet = workbook.add_worksheet(sheet_name)

    font_style = 'Consolas'
    header_format = workbook.add_format({'bold': True, 'align': 'left', 'font_size': 14, 'font_name': font_style, 'text_wrap': True, 'align': 'center'})
    format_number = workbook.add_format({'font_name': font_style, 'font_size': 13, 'num_format': '#,##0'})
    format_percent = workbook.add_format({'num_format': '0%;-0%;0%','font_name': font_style, 'font_size': 13,})
    format_percent_neg = workbook.add_format({'num_format': '0%;-0%;0%','font_name': font_style, 'font_size': 13,'bg_color':   '#FFC7CE','font_color': '#9C0006'})
    format_percent_pos = workbook.add_format({'num_format': '0%;-0%;0%','font_name': font_style, 'font_size': 13,'bg_color':   '#C6EFCE','font_color': '#006100'})
    format_text = workbook.add_format({'font_name': font_style, 'font_size': 13})

    df_tolist = df.values.tolist()
    df_tolist.insert(0, df.columns.tolist())

    # write the data
    for i_x, row in enumerate(df_tolist):
        for i_y, value in enumerate(row):
            if i_x == 0:
                worksheet.write(i_x, i_y, value, header_format)
            worksheet.write(i_x, i_y, value)

    # format the data
    cols_minus_freeze_panes = [i for i in range(df.shape[1])][2:]
    for i in cols_minus_freeze_panes:
        if i % 2 == 1:
            # number columns
            worksheet.set_column(
                f'{xl_col_to_name(i)}:{xl_col_to_name(i)}', 
                width=20, 
                cell_format=format_number
                )
        elif i == 2:
            # rank column
            worksheet.set_column(
                f'{xl_col_to_name(i)}:{xl_col_to_name(i)}', 
                width=20, 
                cell_format=format_number
                )
        else:
            # percent change columns
            worksheet.set_column(
                f'{xl_col_to_name(i)}:{xl_col_to_name(i)}', 
                width=15, 
                cell_format=format_percent
                )
            worksheet.conditional_format(f'{xl_col_to_name(i)}2:{xl_col_to_name(i)}101', {'type': 'cell','criteria': '>=','value': 0,'format':format_percent_pos})
            worksheet.conditional_format(f'{xl_col_to_name(i)}2:{xl_col_to_name(i)}101', {'type': 'cell','criteria': '<','value': 0,'format':format_percent_neg})
            
    worksheet.add_table(
        f"A1:O{df.shape[0]+1}", {
            'header_row': True,
            'autofilter': False,
            'banded_rows': False,
            'columns': [{'header': col} for col in df.columns]
        }
    )

    # # set freeze panes 
    worksheet.freeze_panes(1, 1)

    # # artist song column set
    worksheet.set_column('A:B', width=50, cell_format=format_text)

    # # # billboard chart rank column
    # worksheet.set_column('C', width=30)

    # set header row
    worksheet.set_row(0, cell_format = header_format, height=75)

    workbook.close()

In [ ]:
# data = {"week_id": '202243'}
# headers = {'Content-Type':'application/json','Authorization': 
#             access_token,'x-api-key': api_key,
#             'Accept':'application/vnd.mrc-data.chart.v1+json'}
# url = f'https://api.musicconnect.mrc-data.com/api/chart/data/CHRISTIANAIRPLAY'
# r = requests.post(url=url, headers=headers, data=json.dumps(data))
# r.json()

In [ ]:
current_global_week = get_current_global_week()
current_global_week

# Times
- `202246` 9 min
- `202247` 5 min
- `202248` 5 min 

In [39]:
%%time

access_token = get_authorize_token()


# current_global_week = int(f'{2022}{40:02d}')
# current_global_week = get_current_global_week()

# print(current_global_week)


out_path = Path(f'./outputs/{current_global_week}_luminate_christian_radio_NEW.xlsx').absolute()
chart_df = get_chart(global_week=current_global_week, access_token=access_token)

chart_artist_df = get_artist_data(chart_df, access_token=access_token)
chart_artist_df_cleaned = drop_extra_and_sort_df(chart_artist_df)       
format_and_write_to_excel(chart_artist_df_cleaned, out_path=out_path)

CPU times: user 44 ms, sys: 4.24 ms, total: 48.2 ms
Wall time: 1.29 s


christianairplay                                   \
                 artist_id label rank                  song   
title_id                                                      
174698450           273416          1          Build A Boat   
174171922            72386          2       Perfectly Loved   
177482570         62016067          3  God Is In This Story   
155612371           112061  SPAR    4          New Creation   
177826997            85561          5          The Goodness   
...                    ...   ...  ...                   ...   
157903022         31996152         97  Pour Your Spirit Out   
160365639         13000163  PROV   98    Joy (To The World)   
187081579            10767         99          The Elf Song   
155259842           115205        100          Living Water   
170645186         55554519        101              Hey Girl   

                                                                         \
                                         artist tp spins +/- chg. spins   
title_id                                                                  
174698450                          Colton Dixon      664         -67.00   
174171922           Rachael Lampa Feat. Tobymac      603        -105.00   
177482570        Katy Nichole & Big Daddy Weave      597         -75.00   
155612371                            Mac Powell      551         -68.00   
177826997                               Tobymac      553         -74.00   
...                                         ...      ...            ...   
157903022                        Thrive Worship       30          -1.00   
160365639  Life.Church Worship Feat. Ryan Ellis       23          -9.00   
187081579                               Crowder       16          -7.00   
155259842                         Shane & Shane       28           0.00   
170645186                           Anne Wilson       88         -12.00   

                                                                            \
          lp spins stations adds tp audience +/- chg. audience lp audience   
title_id                                                                     
174698450   731.00    21/41    0     2085800        -289500.00  2375300.00   
174171922   708.00    22/41    0     1961000        -238900.00  2199900.00   
177482570   672.00    22/41    0     1913000        -208900.00  2121900.00   
155612371   619.00    21/41    0     1834200        -176300.00  2010500.00   
177826997   627.00    20/41    0     1804100        -214000.00  2018100.00   
...            ...      ...  ...         ...               ...         ...   
157903022    31.00     9/41    0      100100          -4700.00   104800.00   
160365639    32.00     5/41    0       98300         -96800.00   195100.00   
187081579    23.00     9/41    0       96000         -72300.00   168300.00   
155259842    28.00     8/41    0       94100          30200.00    63900.00   
170645186   100.00     5/41    0       94000         -49300.00   143300.00   

                     
          tp stream  
title_id             
174698450   1142044  
174171922    972276  
177482570   1063850  
155612371    216991  
177826997    637185  
...             ...  
157903022      8268  
160365639     29756  
187081579    377842  
155259842     74681  
170645186    258935  

[100 rows x 14 columns]

# Emailer

In [40]:
from appscript import app, k
from mactypes import Alias
from pathlib import Path

def create_message_with_attachment():
    subject = f'{current_global_week} Global Week Radio Report'
    body = ''
    to_recip = ['Kenny.Rodgers@umusic.com', 'Elizabeth.Brock@umusic.com']

    msg = Message(subject=subject, body=body, to_recip=to_recip, cc_recip=['David.Sylvester@umusic.com', 'Carlos.Monnaco@umusic.com'])

    # attach file
    p = str(out_path)
    msg.add_attachment(p)

    msg.show()

class Outlook(object):
    def __init__(self):
        self.client = app('Microsoft Outlook')

class Message(object):
    def __init__(self, parent=None, subject='', body='', to_recip=[], cc_recip=[], show_=True):

        if parent is None: parent = Outlook()
        client = parent.client

        self.msg = client.make(
            new=k.outgoing_message,
            with_properties={k.subject: subject, k.content: body})

        self.add_recipients(emails=to_recip, type_='to')
        self.add_recipients(emails=cc_recip, type_='cc')

        if show_: self.show()

    def show(self):
        self.msg.open()
        self.msg.activate()

    def add_attachment(self, p):
        # p is a Path() obj, could also pass string

        p = Alias(str(p)) # convert string/path obj to POSIX/mactypes path

        attach = self.msg.make(new=k.attachment, with_properties={k.file: p})

    def add_recipients(self, emails, type_='to'):
        if not isinstance(emails, list): emails = [emails]
        for email in emails:
            self.add_recipient(email=email, type_=type_)

    def add_recipient(self, email, type_='to'):
        msg = self.msg

        if type_ == 'to':
            recipient = k.to_recipient
        elif type_ == 'cc':
            recipient = k.cc_recipient

        msg.make(new=recipient, with_properties={k.email_address: {k.address: email}})

In [41]:
create_message_with_attachment()

# Rodgers, Kenny <Kenny.Rodgers@umusic.com>; Sylvester, David <David.Sylvester@umusic.com>; Brock, Elizabeth <Elizabeth.Brock@umusic.com>; Monnaco, Carlos <Carlos.Monnaco@umusic.com>

# Below for iterating historic

- stuff
- stuff 2

In [ ]:
from requests.exceptions import ReadTimeout

global_weeks = [i for i in range(1, 54)]

# year = datetime.now().strftime('%Y')

year = '2022'
# error = False

for week in global_weeks:

    try:

        current_global_week = int(f'{year}{week:02d}')

        print(current_global_week)

        out_path = Path(f'./outputs/{current_global_week}_luminate_christian_radio.xlsx').absolute()
        chart_df = get_chart(global_week=current_global_week, access_token=access_token)
        chart_artist_df = get_artist_data(chart_df, access_token=access_token)
        chart_artist_df_cleaned = drop_extra_and_sort_df(chart_artist_df)       
        format_and_write_to_excel(chart_artist_df_cleaned, out_path=out_path)

    except ReadTimeout as E:

        print(f'ERROR: {E}, retrying...')

        time.sleep(5)

        current_global_week = int(f'{year}{week:02d}')
        out_path = Path(f'./outputs/{current_global_week}_luminate_christian_radio.xlsx').absolute()
        chart_df = get_chart(global_week=current_global_week, access_token=access_token)
        chart_artist_df = get_artist_data(chart_df, access_token=access_token)
        chart_artist_df_cleaned = drop_extra_and_sort_df(chart_artist_df)
                      
        format_and_write_to_excel(chart_artist_df_cleaned, out_path=out_path)
